# Module 1. Personalize User-Item Interaction Data 준비 및 환경 설정


## Library Import 

파이썬에는 광범위한 라이브러리 모음이 포함되어 있으며, 본 LAB을 위해서 핵심 Data Scientist용 Tool 인 boto3 (AWS SDK) 및 Pandas/Numpy와 같은 라이브러리를 가져와야 합니다. 

In [ ]:
# Imports
#!pip install jsonlines
#from codes import lambda_personalize
import boto3
import json
import numpy as np
import pandas as pd
import time
import jsonlines
import os

from datetime import datetime
import sagemaker
import time
import warnings

import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
import matplotlib.dates as mdate
from botocore.exceptions import ClientError

다음으로 여러분의 환경이 Amazon Personalize와 성공적으로 통신할 수 있는지 확인해야 합니다.

In [ ]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')
s3 = boto3.resource('s3')

생성할 오브젝트의 끝에 임의의 숫자를 부여하기 위해 suffix 정의

In [ ]:
# If you don't need profile, set USER_PROFILE=""
PROFILE_NAME="CUSTOMER"


WORK_DATE="20200716"
WORK_DATE_PATH="2020/07/16"

data_dir = "data"
account_id = "870180618679"

BUCKET_NAME = "customer-poc"
KEY_PREFIX=WORK_DATE


INTERACTION_FILE="interaction.csv"
USER_FILE="user_meta.csv"
ITEM_FILE="item_meta.csv"

DATA_BUCKET_NAME = "customer-poc"
bucket=DATA_BUCKET_NAME 
PREFIX='hrnn-meta'
#bucket_name = DATA_BUCKET_NAME
ROLE_NAME = "CUSTOMER-PersonalizeRetailRole-0612"
suffix = str(np.random.uniform())[4:9]

DATA_SET_GROUP_NAME= WORK_DATE+ "-dataset-group-"+suffix
#SCHEMA_NAME_INTERACTIONS= WORK_DATE+ "-schema-interactions"
#SCHEMA_NAME_USERS= PREFIX + "-schema-users"
#SCHEMA_NAME_ITEMS= PREFIX + "-schema-items"

DATASET_NAME_INTERACTION="customer-dataset-interactions"
DATASET_NAME_USERS="customer-dataset-users"
DATASET_NAME_ITEMS="customer-dataset-items"

suffix = str(np.random.uniform())[4:9]

BATCH_TARGET_USERS_FILE= "users_for_batch_"+ WORK_DATE + ".json"
role_arn="arn:aws:iam::870180618679:role/customer-PersonalizeRetailRole-0612"

DATA_PREFIX = "dataset"
BATCH_PREFIX= "batch"

LOCAL_CLEAN_DATA_PATH=data_dir+'/'+WORK_DATE+'/'+PREFIX+'/'+DATA_PREFIX 
S3_DATA_IMPORT_PATH=WORK_DATE+'/'+PREFIX+'/'+DATA_PREFIX 

S3_BATCH_INPUT_PATH=WORK_DATE+'/'+PREFIX+'/'+BATCH_PREFIX 
S3_BATCH_OUTPUT_PATH=WORK_DATE+'/'+PREFIX+'/'+BATCH_PREFIX 
S3_BATCH_OUTPUT_FILE=S3_BATCH_OUTPUT_PATH+'/'+BATCH_TARGET_USERS_FILE+".out"
LOCAL_BATCH_PATH=data_dir+'/'+WORK_DATE+'/'+PREFIX+'/'+BATCH_PREFIX

LOCAL_BATCH_INPUT_FILE=LOCAL_BATCH_PATH+'/'+BATCH_TARGET_USERS_FILE
LOCAL_BATCH_OUTPUT_FILE=LOCAL_BATCH_PATH+'/'+BATCH_TARGET_USERS_FILE+".out"


os.makedirs(LOCAL_CLEAN_DATA_PATH,exist_ok=True)
os.makedirs(LOCAL_BATCH_PATH,exist_ok=True)

## Interaction Data 다운로드 

In [ ]:
file_name=os.path.join(data_dir,INTERACTION_FILE)

df=pd.read_csv(file_name)
##########################################################
# columns이름 변경 
##########################################################
df.columns=["USER_ID","ITEM_ID","EVENT_TYPE","TIMESTAMP"]
##########################################################
df_origin=df
df.head()


In [ ]:
df=df_origin.copy()
df.dtypes

In [ ]:
df.info()

### NULL DATA제거 

In [ ]:
##########################################################
# columns 이름 변경 df['OOOOO']
##########################################################

print("# of rows of Null TimeStamp : {}".format(df['TIMESTAMP'].isnull().sum()))
print("# of rows of Null Event Type : {}".format(df['EVENT_TYPE'].isnull().sum()))
print("# of rows of Null User ID : {}".format(df['USER_ID'].isnull().sum()))
print("# of rows of Null Item ID : {}".format(df['ITEM_ID'].isnull().sum()))
#df[df['TIMESTAMP'].isnull()]

#########전체 INTERACTION 'null' 확인##########
#row_has_NaN = is_NaN.any(axis=1)
#rows_with_NaN = df[row_has_NaN]
#print(rows_with_NaN)
#len(rows_with_NaN)
##############################################

In [ ]:
df=df.dropna()
df.info()

### 데이터 타입 확인 및 변경

In [ ]:
##############################################
# Time stamp 가 표준 양식대로 프린트 되는지 확인 
# df.TIMESTAMP / 1000000 (분모값 수정)
# 나눠주는 숫자는 input에 따라 다름 
##############################################
그게에 따라 나눠주는 숫자 변경 필요. 
arb_time_stamp = df.iloc[50]['TIMESTAMP']/1000000
print(arb_time_stamp)
print(datetime.utcfromtimestamp(arb_time_stamp).strftime('%Y-%m-%d %H:%M:%S'))

In [ ]:
df.TIMESTAMP  = df.TIMESTAMP / 1000000
arb_time_stamp = df.iloc[50]['TIMESTAMP']
print(arb_time_stamp)
print(datetime.utcfromtimestamp(arb_time_stamp).strftime('%Y-%m-%d %H:%M:%S'))

In [ ]:

print("from",datetime.utcfromtimestamp(df['TIMESTAMP'].min()).strftime('%Y-%m-%d %H:%M:%S'), "to", datetime.utcfromtimestamp(df['TIMESTAMP'].max()).strftime('%Y-%m-%d %H:%M:%S') )

In [ ]:
df.EVENT_TYPE = df.EVENT_TYPE .astype(str)
df.ITEM_ID = df.ITEM_ID.astype(str)
df.USER_ID = df.USER_ID.astype(str)
df.TIMESTAMP=df.TIMESTAMP.astype('int64')

In [ ]:
df.tail()

In [ ]:
#item_unique=pd.DataFrame(item_unique,columns=['ITEM_ID'])
#user_unique=pd.DataFrame(user_unique,columns=['USER_ID'])
item_unique=df.ITEM_ID.unique()
user_unique=df.USER_ID.unique()
item_unique=pd.DataFrame(item_unique)
user_unique=pd.DataFrame(user_unique)
item_unique.to_csv(data_dir+"/" +"unique_item_index.csv")
user_unique.to_csv(data_dir+"/" +"unique_user_index.csv")

###  INTERACTION DATA 정보 확인

Personalize에서 학습을 수행하기 위해서는 다음과 [official limits](https://docs.aws.amazon.com/personalize/latest/dg/limits.html)같은 데이터 요구사항을 맞추어야 합니다. 

* 최소 25명 고유 사용자 
* 최소 100개 고유 아이템 
* 사용자 당 2개 이상의 Interaction(예. 구매,평가 등) 기록


하지만 일반적으로 다음과 같은 데이터가 준비 되어 있는것이 좋습니다. 

* 최소 50명 고유 사용자 
* 최소 100개 고유 아이템 
* 사용자 당 24 이상의 Interaction(예. 구매,평가 등) 기록

In [ ]:
unique_items = df['ITEM_ID'].unique()
unique_users = df['USER_ID'].unique()
print("unique_items: {}, Unique_users : {}".format(len(unique_items),len(unique_users)))

In [ ]:
#for col in category_feature:
#    print("%s column is processing" % (col))
print(df.EVENT_TYPE.value_counts())
df.EVENT_TYPE.value_counts().plot(kind='bar')
plt.title("EVENT_TYPE")

plt.show()

In [ ]:
user_activity_counts = df.groupby("USER_ID").count().loc[:,["EVENT_TYPE"]].rename(columns={"EVENT_TYPE":"INTERACTION_COUNTS"})
print(user_activity_counts.info())
user_activity_counts.head()


In [ ]:
user_activity_counts.quantile([.1,.2,.3,.4,.5,.6,.8,.9,.95,.99,.999,.9999,1.0])

In [ ]:
user_activity_counts=user_activity_counts.reset_index()
activities = user_activity_counts.groupby('INTERACTION_COUNTS').count()
activities.columns=['NUM_USERS']
activities

In [ ]:
assert (len(unique_users)==activities['NUM_USERS'].sum())

In [ ]:
activities.loc[:,:].plot(kind='bar', figsize=(15,5), ylim=(0,activities.index.max()))
plt.title("activities users group")

plt.show()

In [ ]:
print("Users with interactions more than 24 interactions: {}".format(activities[activities.index > 24].NUM_USERS.sum()))
print("Users with interactions less than & equal 24 interactions: {}".format(activities[activities.index <= 24].NUM_USERS.sum()))
slow_user=user_activity_counts[user_activity_counts['INTERACTION_COUNTS']<=24]
print("slow_user_percentage:{}".format(len(slow_user)/len(user_activity_counts)))

In [ ]:
df.groupby("USER_ID").count().mean()

### 아이템 분석

In [ ]:
def draw_long_tails(df, is_in_event_types, groupby_column_name, count_name, range1=10, range2=100, range3=300):
    filtered_df = df.loc[df["EVENT_TYPE"].isin(is_in_event_types)]
    groupby_df = filtered_df.groupby(groupby_column_name).count().loc[:,["EVENT_TYPE"]].rename(columns={"EVENT_TYPE":count_name})
    sorted_df = groupby_df.sort_values([count_name], ascending=False)
    
    sorted_df[:].plot(kind='line', figsize=(15,5))
    plt.title(count_name + " per " + groupby_column_name)
    plt.show()

    sorted_df[:range1].plot(kind='line', figsize=(15,5))
    plt.title(count_name + " per " + groupby_column_name + "/ 1 ~" + str(range1) + "th")
    plt.show()
    
    sorted_df[range1:range2].plot(kind='line', figsize=(15,5))
    plt.title(count_name + " per " + groupby_column_name + " / " + str(range1) + "th ~ "+ str(range2) + "th")
    plt.show()

    sorted_df[range2:range3].plot(kind='line', figsize=(15,5))
    plt.title(count_name + " per " + groupby_column_name + " / " + str(range2) + "th ~ "+ str(range3) + "th")
    plt.show()
    
    sorted_df[range3:].plot(kind='line', figsize=(15,5))
    plt.title(count_name + " per " + groupby_column_name + " / " + str(range3) + "th ~ ")
    plt.show()
    
#draw_long_tails(interactions_org, ["order"], "ITEM_ID", "ORDERS", 100, 1000, 3000)
draw_long_tails(df, ["order","click"], "ITEM_ID", "ORDERS & CLICK", 10, 100, 300)

## User Data 분석

In [ ]:
file_name=os.path.join(data_dir,USER_FILE)
users=pd.read_csv(file_name)
users.columns=['USER_ID','AGE','GENDER']
users.head()


In [ ]:
users.USER_ID = users.USER_ID.astype(str)
users.AGE = users.AGE.astype(str)
users.GENDER = users.GENDER.astype(str)
users.info()

In [ ]:
user_gender_counts = users.groupby('GENDER').count().loc[:,["USER_ID"]].rename(columns={"USER_ID":"USER_GENDER_COUNTS"})
user_gender_counts

In [ ]:
user_gender_counts = users.groupby('AGE').count().loc[:,["USER_ID"]].rename(columns={"USER_ID":"USER_AGE_COUNTS"})
user_gender_counts

## Item 데이터 분석 

In [ ]:
file_name=os.path.join(data_dir,ITEM_FILE)
items=pd.read_csv(file_name)
items.columns=["ITEM_ID","CATEGORY"]
items.head()


In [ ]:
items.info()

In [ ]:
item_cat_counts = items.groupby('CATEGORY').count().loc[:,["ITEM_ID"]].rename(columns={"ITEM_ID":"CATEGORY_COUNTS"})
item_cat_counts 

In [ ]:
print("# of rows of Null ITEM_ID : {}".format(items['ITEM_ID'].isnull().sum()))
print("# of rows of Null CATEGORY : {}".format(items['CATEGORY'].isnull().sum()))

In [ ]:
items.ITEM_ID = items.ITEM_ID.astype(str)
items.CATEGORY = items.CATEGORY.astype(str)

### 데이터 S3 업로드 

In [ ]:
# File upload에 몇분 소요 (600MB)

interaction_object_key = "interactions_" + WORK_DATE + ".csv"
user_object_key="users_"+ WORK_DATE + ".csv"
item_object_key="items_"+WORK_DATE + ".csv"

interaction_file=LOCAL_CLEAN_DATA_PATH+"/"+interaction_object_key
user_file=LOCAL_CLEAN_DATA_PATH+"/"+user_object_key
item_file=LOCAL_CLEAN_DATA_PATH+"/"+item_object_key

df[["USER_ID","ITEM_ID","TIMESTAMP","EVENT_TYPE"]].to_csv(interaction_file,index=False)
users.to_csv(user_file,index=False)
items.to_csv(item_file,index=False)


In [ ]:
S3_DATA_IMPORT_PATH=WORK_DATE+'/'+PREFIX+'/'+DATA_PREFIX 

target=S3_DATA_IMPORT_PATH+"/"+interaction_object_key
boto3.Session().resource('s3').Bucket(DATA_BUCKET_NAME).Object(target).upload_file(interaction_file)

target=S3_DATA_IMPORT_PATH+"/"+user_object_key
boto3.Session().resource('s3').Bucket(DATA_BUCKET_NAME).Object(target).upload_file(user_file)


target=S3_DATA_IMPORT_PATH+"/"+item_object_key
boto3.Session().resource('s3').Bucket(DATA_BUCKET_NAME).Object(target).upload_file(item_file)

In [ ]:
print(df.head(2))
print(items.head(2))
print(users.head(2))

## 스키마 생성

Personalize가 데이터를 이해하는 방법의 핵심 구성 요소는 아래 정의 된 스키마(schema)에서 비롯됩니다. 이 설정은 CSV 파일을 통해 제공된 데이터를 요약하는 방법을 Personalize 서비스에 알려줍니다. 열(column)과 유형(type)은 위에서 만든 파일의 내용과 일치합니다.

### Interaction 스키마 생성

In [ ]:
interaction_schema_name="Interaction-schema-"+WORK_DATE+"-"+suffix

In [ ]:
interaction_schema = {
    "type": "record",
    "name": "Interactions",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "USER_ID",
            "type": "string"
        },
        {
            "name": "ITEM_ID",
            "type": "string"
        },
       
        
        { 
            "name": "EVENT_TYPE",
            "type": "string"
        },
        {
            "name": "TIMESTAMP",
            "type": "long"
        }
    ],
    "version": "1.0"
}


create_schema_response = personalize.create_schema(
    name = interaction_schema_name,
    schema = json.dumps(interaction_schema)
)

interaction_schema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))

### User 스키마 생성

In [ ]:
user_schema_name="User-schema-"+WORK_DATE+"-"+suffix

In [ ]:
user_schema = {
    "type": "record",
    "name": "Users",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "USER_ID",
            "type": "string"
        },
      {
          "name": "AGE",
          "type": "string",
          "categorical": True
      },
        {
          "name": "GENDER",
          "type": "string",
          "categorical": True
      }
     
    ],
    "version": "1.0"
}


create_schema_response = personalize.create_schema(
    name = user_schema_name,
    schema = json.dumps(user_schema)
)

user_schema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))

In [ ]:
item_schema_name="Item-schema-"+WORK_DATE+"-"+suffix

In [ ]:
item_schema = {
    "type": "record",
    "name": "Items",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
    {
        "name": "ITEM_ID",
        "type": "string"
    },
   
    
    {
        "name": "CATEGORY",
        "type": ["string","null"],
        "categorical": True
    }
    
      
        
    ],
    "version": "1.0"
}

create_metadata_schema_response = personalize.create_schema(
    name = item_schema_name,
    schema = json.dumps(item_schema)
)

item_schema_arn = create_metadata_schema_response['schemaArn']
print(json.dumps(create_metadata_schema_response, indent=2))


## 데이터 세트 그룹 생성 및 대기

Personalize에서 가장 큰 단위는 **데이터 세트 그룹(Dataset Group)** 이며, 이렇게 하면 데이터, 이벤트 추적기(event tracker), 솔루션(solution) 및 캠페인(campaign)이 분리됩니다. 공통의 데이터 수집을 공유하는 것들을 그룹화합니다. 원하는 경우 아래 그룹명을 자유롭게 변경해 주세요.

### 데이터 세트 그룹 생성

In [ ]:
create_dataset_group_response = personalize.create_dataset_group(
    name = DATA_SET_GROUP_NAME
)

dataset_group_arn = create_dataset_group_response['datasetGroupArn']
print(json.dumps(create_dataset_group_response, indent=2))

#### 데이터 세트 그룹이 활성화 상태가 될 때까지 대기

아래의 모든 항목에서 Dataset Group을 사용하려면 활성화(active)가 되어야 합니다. 아래 셀을 실행하고 DatasetGroup: ACTIVE로 변경될 때까지 기다려 주세요.

In [ ]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_group_response = personalize.describe_dataset_group(
        datasetGroupArn = dataset_group_arn
    )
    status = describe_dataset_group_response["datasetGroup"]["status"]
    print("DatasetGroup: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(15)

### 데이터 세트 생성

그룹 다음으로 생성할 것은 실제 데이터 세트입니다. 아래의 코드 셀을 실행하여 데이터 세트을 생성해 주세요.

### Interaction 데이터 세트 생성

In [ ]:
dataset_type = "INTERACTIONS"
create_dataset_response = personalize.create_dataset(
    name = DATASET_NAME_INTERACTION,
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = interaction_schema_arn
)

interaction_dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

###  USER 데이터 세트 생성 

In [ ]:
dataset_type = "USERS"

create_dataset_response = personalize.create_dataset(
    name = DATASET_NAME_USERS,
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = user_schema_arn
)

user_dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

### ITEM 데이터 세트 생성

In [ ]:
dataset_type = "ITEMS"

create_dataset_response = personalize.create_dataset(
    name = DATASET_NAME_ITEMS,
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = item_schema_arn
)

item_dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

### S3 버킷에 정책 부여

Amazon Personalize는 앞서 생성한 S3 버킷의 내용을 읽을 수 있어야 합니다. 아래 코드 셀로 S3 버킷 접근 정책(policy)을 부여합니다.

In [ ]:
s3 = boto3.client("s3")

policy = {
    "Version": "2012-10-17",
    "Id": "PersonalizeS3BucketAccessPolicy",
    "Statement": [
        {
            "Sid": "PersonalizeS3BucketAccessPolicy",
            "Effect": "Allow",
            "Principal": {
                "Service": "personalize.amazonaws.com"
            },
            "Action": [
                "s3:*Object",
                "s3:ListBucket",
            ],
            "Resource": [
                "arn:aws:s3:::{}".format(bucket),
                "arn:aws:s3:::{}/*".format(bucket)
            ]
        }
    ]
}

s3.put_bucket_policy(Bucket=DATA_BUCKET_NAME, Policy=json.dumps(policy))

### Personalize IAM Role 생성

또한, Amazon Personalize는 특정 작업들을 실행할 권한을 갖기 위해, AWS에서 역할을 맡을 수 있는 기능이 필요합니다. 

In [ ]:
iam = boto3.client("iam")

role_name = ROLE_NAME
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
          "Effect": "Allow",
          "Principal": {
            "Service": "personalize.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
        }
    ]
}




try:
    create_role_response = iam.create_role(
        RoleName = role_name,
        AssumeRolePolicyDocument = json.dumps(assume_role_policy_document)
    );

    iam.attach_role_policy(
        RoleName = role_name,
        PolicyArn = "arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"
    );

    role_arn = create_role_response["Role"]["Arn"]
except ClientError as e:
    if e.response['Error']['Code'] == 'EntityAlreadyExists':
        role_arn = iam.get_role(RoleName=role_name)['Role']['Arn']
    else:
        raise
        

# AmazonPersonalizeFullAccess provides access to any S3 bucket with a name that includes "personalize" or "Personalize" 
# if you would like to use a bucket with a different name, please consider creating and attaching a new policy
# that provides read access to your bucket or attaching the AmazonS3ReadOnlyAccess policy to the role
policy_arn = "arn:aws:iam::aws:policy/service-role/AmazonPersonalizeFullAccess"
iam.attach_role_policy(
    RoleName = role_name,
    PolicyArn = policy_arn
)

# Now add S3 support
iam.attach_role_policy(
    RoleName=role_name,    
    PolicyArn='arn:aws:iam::aws:policy/AmazonS3FullAccess'
)
time.sleep(60) # wait for a minute to allow IAM role policy attachment to propagate


### 데이터 세트 Import

이전에는 정보를 저장하기 위해 데이터 세트 그룹 및 데이터 세트를 생성했으므로, 
이제는 모델 구축을 위해 S3에서 Amazon Personalize로 데이터를 로드하는 import job을 실행합니다.

#### Interaction 데이터 세트 Import Job 생성

In [ ]:
create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "customer-interaction-dataset-import-" + WORK_DATE+"-"+suffix,
    datasetArn = interaction_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(DATA_BUCKET_NAME, S3_DATA_IMPORT_PATH+"/"+interaction_object_key)
    },
    roleArn = role_arn
)

interation_dataset_import_job_arn = create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))

In [ ]:
create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "customer-users-dataset-import-" + WORK_DATE+"-"+suffix,
    datasetArn = user_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(DATA_BUCKET_NAME, S3_DATA_IMPORT_PATH+"/"+user_object_key)
    },
    roleArn = role_arn
)

user_dataset_import_job_arn = create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))

In [ ]:
create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "customer-item-dataset-import-" + WORK_DATE+"-"+suffix,
    datasetArn = item_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(DATA_BUCKET_NAME, S3_DATA_IMPORT_PATH+"/"+item_object_key)
    },
    roleArn = role_arn
)

item_dataset_import_job_arn = create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))

In [ ]:
#store values 세션 끊길거 대비 
%store BUCKET_NAME
%store DATA_BUCKET_NAME
%store dataset_group_arn
%store interaction_dataset_arn
%store user_dataset_arn
%store item_dataset_arn
%store interation_dataset_import_job_arn
%store user_dataset_import_job_arn
%store item_dataset_import_job_arn
%store ROLE_NAME
%store role_arn

#### 아이템 데이터 세트 Import Job 생성

### 데이터 세트 Import job이 활성화 상태가 될 때까지 대기

Import job이 완료되기까지 시간이 걸립니다. 아래 코드 셀의 출력 결과가 DatasetImportJob: ACTIVE가 될 때까지 기다려 주세요.

In [ ]:
%%time

status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = interation_dataset_import_job_arn
    )
    
    dataset_import_job = describe_dataset_import_job_response["datasetImportJob"]
    if "latestDatasetImportJobRun" not in dataset_import_job:
        status = dataset_import_job["status"]
        print("DatasetImportJob: {}".format(status))
    else:
        status = dataset_import_job["latestDatasetImportJobRun"]["status"]
        print("LatestDatasetImportJobRun: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

## 솔루션 생성 

#### HRNN

In [ ]:
 # Recipe 선택 
hrnn_recipe_arn = "arn:aws:personalize:::recipe/aws-hrnn-metadata"

In [ ]:
# Solution 생성 
create_solution_response = personalize.create_solution(
    name = "customer-hrnn-meta-all" + WORK_DATE,
    datasetGroupArn = dataset_group_arn,
    recipeArn = hrnn_recipe_arn,
)

hrnn_meta_solution_arn = create_solution_response['solutionArn']
print(json.dumps(create_solution_response, indent=2))

In [ ]:
# 솔루션 버전 생성
create_solution_version_response = personalize.create_solution_version(
    solutionArn = hrnn_meta_solution_arn
)

hrnn_meta_solution_version_arn = create_solution_version_response['solutionVersionArn']
print(json.dumps(create_solution_version_response, indent=2))

In [ ]:
%%time

max_time = time.time() + 8*60*60 # 8 hours
while time.time() < max_time:
      
    #hrnn status
    describe_solution_version_response = personalize.describe_solution_version(
        solutionVersionArn = hrnn_meta_solution_version_arn
    )  
    status_hrnn_meta = describe_solution_version_response["solutionVersion"]["status"]
    print("HRNN META SolutionVersion: {}".format(status_hrnn_meta))
    
    if  (status_hrnn_meta == "ACTIVE" or status_hrnn_meta == "CREATE FAILED"):
        break
 
    time.sleep(300)

print("All solution creation completed")

In [ ]:
%store hrnn_meta_solution_version_arn
%store hrnn_meta_solution_arn

## 솔루션 평가하기 

In [ ]:
%store -r

In [ ]:
get_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = hrnn_meta_solution_version_arn
)

print(json.dumps(get_solution_metrics_response, indent=2))



## Batch Inference

In [ ]:
batch_target_users = df.USER_ID.unique()
n = 0
f = open(LOCAL_BATCH_INPUT_FILE,"w")
for user in batch_target_users:
    f.write("{\"userId\":\"%s\"}\n" % user)
    n = n + 1
    #for the test
    #if n> 100:
    #    break
    
f.close()
print(BATCH_TARGET_USERS_FILE)
print(n)

In [ ]:
# Upload files to S3
boto3.Session().resource('s3').Bucket(DATA_BUCKET_NAME).Object(S3_BATCH_INPUT_PATH+"/"+BATCH_TARGET_USERS_FILE).upload_file(LOCAL_BATCH_INPUT_FILE)

s3_input_path = "s3://" + DATA_BUCKET_NAME + "/"+S3_BATCH_INPUT_PATH+"/"+BATCH_TARGET_USERS_FILE
print(s3_input_path)

In [ ]:
# Define the output path
s3_output_path = "s3://" +DATA_BUCKET_NAME +"/"+S3_BATCH_OUTPUT_PATH+"/"
print(s3_output_path)

In [ ]:
batchInferenceJobArn = personalize.create_batch_inference_job (
    solutionVersionArn = hrnn_meta_solution_version_arn,
    jobName = "Customer-Batch-Inference-Job-HRNN-META-"+WORK_DATE+"-"+suffix,
    roleArn = role_arn,
    jobInput = 
     {"s3DataSource": {"path": s3_input_path}},
    jobOutput = 
     {"s3DataDestination":{"path": s3_output_path}}
)
batchInferenceJobArn = batchInferenceJobArn['batchInferenceJobArn']
batchInferenceJobArn

In [ ]:
current_time = datetime.now()
print("Import Started on: ", current_time.strftime("%I:%M:%S %p"))

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_inference_job_response = personalize.describe_batch_inference_job(
        batchInferenceJobArn = batchInferenceJobArn
    )
    status = describe_dataset_inference_job_response["batchInferenceJob"]['status']
    print("HRNN Meta DatasetInferenceJob: {}".format(status))
    
    describe_dataset_inference_job_response = personalize.describe_batch_inference_job(
        batchInferenceJobArn = batchInferenceJobArn
    )
  
    if (status == "ACTIVE" or status == "CREATE FAILED"):
        break
        
    time.sleep(60)
    
current_time = datetime.now()
print("Import Completed on: ", current_time.strftime("%I:%M:%S %p"))

In [ ]:
s3 = boto3.client('s3')
s3.download_file(DATA_BUCKET_NAME,S3_BATCH_OUTPUT_FILE,LOCAL_BATCH_OUTPUT_FILE)


In [ ]:

def read_batch_result(filename, key_name):
    result = {}
    fp = open(filename)
    reader = jsonlines.Reader(fp)
    count = 0
    for record in reader:
        result[record["input"][key_name]] = str.join(",",record["output"]["recommendedItems"])
        count += 1
    reader.close()
    fp.close()
    
    print("Found records:", count, len(result))
    return result

def unique_count(result):
    recommend_keys = {}
    
    duplicate_cnt = 0
    unique_cnt = 0
    for k in result:
        key = result[k]
        
        if key not in recommend_keys:
            unique_cnt += 1            
            recommend_keys[key] = { "count": 1, "keys": k }
        else:
            recommend_keys[key]["count"] = recommend_keys[key]["count"] + 1
            #recommend_keys[key]["keys"] = recommend_keys[key]["keys"] + "," + k
            duplicate_cnt += 1
        #print(f'dup:%10d, unique:%10d, %s is processing…\r' % (duplicate_cnt, unique_cnt, k), end="")

    return recommend_keys, unique_cnt, duplicate_cnt

hrnn_result = read_batch_result(LOCAL_BATCH_OUTPUT_FILE, "userId")
recommend_keys, unique_cnt, duplicate_cnt = unique_count(hrnn_result)

print(f'dup:%10d, unique:%10d\n' % (duplicate_cnt, unique_cnt))
cnt = 10
for i in sorted(recommend_keys, key=lambda x: (recommend_keys[x]['count']), reverse=True):
    print(recommend_keys[i], i)
    if cnt < 0: 
        break
    cnt -=1

In [ ]:
%store batchInferenceJobArn

## 캠페인 생성하기 

In [ ]:
'''
create_campaign_response = personalize.create_campaign(
    name = "Customer-hrnn-campaign-" + WORK_DATE,
    solutionVersionArn = hrnn_solution_version_arn,
    minProvisionedTPS = 1
)

hrnn_campaign_arn = create_campaign_response['campaignArn']
print(json.dumps(create_campaign_response, indent=2))
'''

In [ ]:
'''
user_id= test_user_id
get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = hrnn_campaign_arn,
    userId = str(user_id),
)
# Update DF rendering
pd.set_option('display.max_rows', 30)

print("Recommendations for user: ", user_id)

item_list = get_recommendations_response['itemList']

recommendation_title_list = []
recommendation_id_list=[]
for item in item_list:
    recommendation_title_list.append(title)
    recommendation_id_list.append(item['itemId'])
recommendations_df = pd.DataFrame(recommendation_title_list, columns = ['OriginalRecs'])
recommendations_df
'''

In [ ]:
%store hrnn_campaign_arn
